In [1]:
#Import the gtfs-kit module
import gtfs_kit as gk
import pandas as pd
import datetime

In [2]:
feeds = []
feed1 = gk.feed.read_feed('../feed_collection/20161210.gtfs',dist_units="km")
feed2 = gk.feed.read_feed('../feed_collection/20170415.gtfs',dist_units="km")
feed3 = gk.feed.read_feed('../feed_collection/20171224.gtfs',dist_units="km")
feed4 = gk.feed.read_feed('../feed_collection/20181211.gtfs',dist_units="km")
feed5 = gk.feed.read_feed('../feed_collection/20191229.gtfs',dist_units="km")
feed6 = gk.feed.read_feed('../feed_collection/20201217.gtfs',dist_units="km")
feed7 = gk.feed.read_feed('../feed_collection/20211214.gtfs',dist_units="km")
feed8 = gk.feed.read_feed('../feed_collection/20221214.gtfs',dist_units="km")
feeds.extend((feed1,feed2,feed3,feed4,feed5,feed6,feed7,feed8))

In [3]:
feed1.shapes.head(20)

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,7970137,43.331867,11.322687,1,0.00000
1,7970137,43.332570,11.320706,2,0.17867
2,7970137,43.332742,11.320156,3,0.22724
3,7970137,43.333431,11.317752,4,0.43666
4,7970137,43.333615,11.317185,5,0.48700
5,7970137,43.333780,11.316749,6,0.52683
6,7970137,43.333937,11.316389,7,0.56089
7,7970137,43.334296,11.315710,8,0.62888
8,7970137,43.334628,11.315202,9,0.68419
9,7970137,43.334850,11.314907,10,0.71861


In [4]:
### Creation of dataset to save
routes = pd.DataFrame()
stop_times = pd.DataFrame()
trips = pd.DataFrame()
stops = pd.DataFrame()
shapes = pd.DataFrame()
calendar_dates = pd.DataFrame()

In [5]:
    shapes_feed = feed1.routes
    print("-" * 10)
    print(shapes_feed)
    print("-" * 10)

----------
      route_id agency_id route_short_name  \
0   1004295345       163              NaN   
1   1013468805       163              NaN   
2   1017917198       163              NaN   
3   1023125973       163              NaN   
4   1038909885       163              NaN   
5   1055743619       163              NaN   
6   1081089062       163              NaN   
7   1085727347       163              NaN   
8   1109125441       163              NaN   
9   1145664474       163              NaN   
10  1236794033       163              NaN   
11  1286685275       163              NaN   
12  1809279468       163              NaN   
13  1891176590       163              NaN   
14  2101556838       163              NaN   

                                   route_long_name  route_type route_color  \
0                         Firenze - Pisa - Livorno           2      FFFFFF   
1   Pisa - Livorno - Grosseto (Rm) (Tirrenica Sud)           2      FFFFFF   
2           (Ge) La Spezia - Pisa 

In [6]:
for feed in feeds:
    routes_feed = gk.routes.get_routes(feed)
    stop_times_feed = feed.get_stop_times()
    trips_feed = feed.get_trips()
    stops_feed = feed.get_stops()
    shapes_feed = feed.shapes
    calendar_dates_feed = feed.calendar_dates
    
    ### Get the route information for Routes where route is Pisa-Lucca-Aulla and usefull ids
    sub_routes = routes_feed.loc[routes_feed['route_long_name'] == 'Pisa - Lucca - Aulla'].reset_index(drop=True)
    routes = pd.concat([routes, sub_routes])
    route_id = sub_routes.iloc[0]['route_id']
    
    ### Get all the trips in for Pisa-Lucca-Aulla
    sub_trips = trips_feed.loc[trips_feed['route_id'] == route_id].reset_index(drop=True)
    trips = pd.concat([trips, sub_trips]).reset_index(drop=True)
    
    ### Get all the stops time regarding the Pisa-Lucca-Aulla 
    sub_stop_times = pd.merge(sub_trips, stop_times_feed, on=['trip_id','trip_id']).reset_index(drop=True)
    sub_stop_times = sub_stop_times.drop(['route_id', 'service_id','trip_headsign', 'trip_short_name','direction_id', 'shape_id'], axis=1)
    stop_times = pd.concat([stop_times, sub_stop_times]).reset_index(drop=True)

    ### Get all the stops of the Pisa-Lucca-Aulla route
    sub_stops = pd.merge(sub_stop_times,stops_feed, on=['stop_id','stop_id']).reset_index(drop=True)
    sub_stops = sub_stops.drop(['trip_id', 'arrival_time','departure_time', 'stop_sequence','shape_dist_traveled'], axis=1)
    stops = pd.concat([stops, sub_stops]).reset_index(drop=True)

    ### Get all the shapes related with the Pisa-Lucca-Aulla trips
    sub_shapes = pd.merge(sub_trips, shapes_feed, on=['shape_id','shape_id']).reset_index(drop=True)
    sub_shapes = sub_shapes.drop(['route_id', 'service_id','trip_headsign', 'trip_short_name','direction_id', 'trip_id'], axis=1)
    shapes = pd.concat([shapes, sub_shapes]).reset_index(drop=True)
    
    ### Get all the calendar dates related with the Pisa-Lucca-Aulla trips
    sub_dates = pd.merge(sub_trips, calendar_dates_feed, on=['service_id','service_id']).reset_index(drop=True)
    sub_dates = sub_dates.drop(['route_id', 'shape_id','trip_headsign', 'trip_short_name','direction_id', 'trip_id'], axis=1)
    calendar_dates = pd.concat([calendar_dates, sub_dates]).reset_index(drop=True)


In [7]:
#Clean of route dataset
routes = routes.drop_duplicates().reset_index(drop=True)
#Clean of trips dataset
trips = trips.drop_duplicates().reset_index(drop=True)
#Clean of stops dataset
stops = stops.drop_duplicates().reset_index(drop=True)
#Clean of stop_times dataset
stop_times = stop_times.drop_duplicates().reset_index(drop=True)
#Clean of shapes dataset
shapes = shapes.drop_duplicates().reset_index(drop=True)
#Clean of calendar_dates dataset
calendar_dates = calendar_dates.drop_duplicates().reset_index(drop=True)

In [8]:
### Drop exception type 
calendar_dates = calendar_dates.drop(['exception_type'], axis=1)

In [9]:
# Creation of calendar dataset
calendar = pd.DataFrame(columns=['service_id','monday','tuesday','wednesday','thursday','friday','saturday','sunday','start_date','end_date'])
# Group by the service id
grouped = calendar_dates.groupby('service_id')
dfg = grouped.aggregate(lambda x: tuple(x)).reset_index(drop=False)
# Iterate through each row and select
# 'Name' and 'Stream' column respectively.
for ind in dfg.index:
    dates = dfg['date'][ind]
    date_list = dates[0:14]
    mon = tue = wed = thu = fri = sat = sun = False
    for show_date_str in date_list:
        date = datetime.datetime.strptime(show_date_str, "%Y%m%d")
        day_of_week_name = date.strftime('%A').lower()
        if day_of_week_name == "monday":
            mon = True
        elif day_of_week_name == "tuesday":
            tue = True
        elif day_of_week_name == "wednesday":
            wed = True
        elif day_of_week_name == "thursday":
            thu = True
        elif day_of_week_name == "friday":
            fri = True
        elif day_of_week_name == "saturday":
            sat = True
        elif day_of_week_name == "sunday":
            sun = True

    calendar.loc[len(calendar.index)] = [dfg['service_id'][ind], mon, tue, wed, thu, fri, sat, sun, dates[0], dates[-1]]     

In [10]:
#Creation of calendar_dates dataset
calendar = calendar.drop_duplicates().reset_index(drop=True)


In [11]:
### Save the preprocessed data as a zip - equal to gtfs.
result = gk.feed.read_feed('../feed_collection/20161210.gtfs',dist_units="km")
result.routes = routes
result.trips = trips
result.stops = stops
result.stop_times = stop_times
result.calendar_dates = calendar_dates
result.calendar = calendar
result.shapes = shapes
result.write('../preprocessing.zip', 9)